In [ ]:
# loading arabicWord2vec model
from gensim.models  import Doc2Vec,Word2Vec
sg_ar_twitter =Word2Vec.load("/content/drive/My Drive/OffensEval 2020/Data/embeddings_skip_grams_100/full_grams_sg_100_twitter.mdl")


In [ ]:
import re
import codecs
def read_data(file_path):
  X = []
  Y = []
  pos_num=0
  neg_num=0
  file1= codecs.open(file_path, 'r', 'utf-8')
  for line in file1:
    if line == "nan" or line == "\n":
      continue
    out = line.split(',')
    X.append(out[0])
    label=out[1]
    if "OFF" in label:
      Y.append(1)
      pos_num+=1
    else:
      Y.append(0)
      neg_num+=1

  return X,Y, pos_num, neg_num# # read data test


# read data test
def read_X_test():
  test=codecs.open('/content/drive/My Drive/OffensEval 2020/Data/offensive_eval_subtask_A/OSACT2020-sharedTask-test-tweets.txt','r','utf-8')
  X_test=[]
  for line in test:
      t = normalize_text(line)
      X_test.append(t)
  test.close()
  print("test data " , len(X_test))
  return X_test


In [ ]:


x_train, y_train, pos_num, neg_num = read_data('/content/drive/My Drive/OffensEval 2020/Data/offensive_eval_subtask_A/taskA_cleaned_stemmed_train_file1.txt')
print(len(y_train))
x_val, y_val, pos_num1, neg_num1 = read_data('/content/drive/My Drive/OffensEval 2020/Data/offensive_eval_subtask_A/taskA_cleaned_stemmed_val_file1.txt')
print(len(y_val))
#x_test = read_X_test()


In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
import numpy as np
# retrun embedding matrix [(all word in copus) ,(dimension for every word)]

def create_embedding_matrix(model, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    i=0
    for word in word_index:
      try :
        idx = word_index[word]
        word_vector = model.wv[word]
        embedding_matrix[idx] = np.array(
                      word_vector, dtype=np.float32)[:embedding_dim]

      except KeyError:
        embedding_matrix[idx]=np.random.normal(0,np.sqrt(0.25),embedding_dim)


    return embedding_matrix

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=50000, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)

tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)

x_val = tokenizer.texts_to_sequences(x_val)
#x_test = tokenizer.texts_to_sequences(X_test)
#X = tokenizer.texts_to_sequences(X)

vocab_size = len(tokenizer.index_word) + 1
print(vocab_size)


embedding_matrix = create_embedding_matrix(sg_ar_twitter, tokenizer.word_index,100)

#del model

In [ ]:
from keras.preprocessing.sequence import pad_sequences
max_len = 50
x_train = pad_sequences(x_train, maxlen=max_len,truncating='post', padding='post', value = 0)
x_val = pad_sequences(x_val, maxlen=max_len,truncating='post', padding='post', value = 0)
#x_test =pad_sequences(X_test, maxlen=max_len,truncating='post', padding='post', value = 0)
#print(x_train)

In [ ]:
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers


from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *
import tensorflow as tf


In [ ]:
def train(x_train , y_train , x_val, y_val , model_obj):
        clf = model_obj
        clf.load_weights('model.h5')
        history = clf.fit(x_train, y_train, batch_size=100, epochs=20, validation_data=(x_val, y_val) )
        # class_weight={1:0.9, 0:0.1}
        return history ,clf


In [ ]:
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)


class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

from keras.layers import Dense, Dropout, Flatten
def model_lstm_atten(embedding_matrix):
    inp = Input(shape=(max_len,))
    print(vocab_size)
    x = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(inp)
    x=Dropout(0.5)(x)
    # note : we can use CuDNNLSTM rethar than CuDNNGRU
    x = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
    x=Dropout(0.5)(x)
    x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    x = AttentionWithContext()(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer='adam',  metrics=['accuracy',precision_m, recall_m, f1_m])
    return model

In [ ]:
model = model_lstm_atten(embedding_matrix)
# Changed it here a little bit since the custom attention layer is not getting deepcopy
model.save_weights('model.h5')

In [ ]:
from sklearn.model_selection import StratifiedKFold
#x_val=[i[0] for i in x_val]
#print(x_train)
#print(y_val)
hist , model_tr = train(x_train , y_train , x_val , y_val,model)

In [ ]:
loss, acc, precesion, recall, f1= model.evaluate(x_val, y_val)
print("Loss: ",loss)
print("acc: ",acc)
print("precesion: ",precesion)
print("recall: ",recall)
print("f1: ",f1)
#y_predict=model.predict(x_test)


In [ ]:
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from torch.optim.optimizer import Optimizer

def bestThresshold(y_train,train_preds):
    tmp = [0,0,0] # idx, cur, max
    delta = 0
    for tmp[0] in tqdm(np.arange(0.1, 0.501, 0.01)):
        tmp[1] = f1_score(y_train, np.array(train_preds)>tmp[0])
        if tmp[1] > tmp[2]:
            delta = tmp[0]
            tmp[2] = tmp[1]
    print('best threshold is {:.4f} with F1 score: {:.4f}'.format(delta, tmp[2]))
    return delta , tmp[2]

valid= model.predict(x_val)
t=[i[0] for i in valid]
delta, _= bestThresshold(y_val,t)
'''
off = 0 ------> best threshold is 0.2300 with F1 score: 0.9574
off = 1 ----- >best threshold is 0.3600 with F1 score: 0.7305
---------------------------------------------------------------------
Hs = 1 ----- >best threshold is 0.1200 with F1 score: 0.3736
Hs = 0 ------>best threshold is 0.2600 with F1 score: 0.9805
'''

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
import pandas as pd
def plot_confusion_matrix(y_test,y_pred, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
  """
  This function prints and plots the confusion matrix.
  Normalization can be applied by setting `normalize=True`.
  """
  clf_report = classification_report(y_test, y_pred, target_names=classes, output_dict=True)
  # .iloc[:-1, :] to exclude support
  sns.heatmap(pd.DataFrame(clf_report).iloc[:-1, :].T, annot=True)

In [ ]:
def predict_with_best_threshold(y_predict , threshold):
  predict=[]
  for i in range(len(y_predict)):
    if y_predict[i] >= threshold :
      predict.append(1)
    else :
      predict.append(0)
  return predict

predicted=model.predict(x_val)

plot_confusion_matrix(y_val, predicted.round(), classes=list(set(y_val)))

In [ ]:
predicted=model.predict(x_val)
pre = predict_with_best_threshold(predicted ,delta)
plot_confusion_matrix(y_val, pre, classes=list(set(y_val)))

In [ ]:
from matplotlib import pyplot as plt
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
model.save('/content/drive/My Drive/OffensEval 2020/Models/attention_model/model_att2_stemming.model')

In [ ]:
## don't read down not completed


In [ ]:
import re
import numpy as np
import random
import os
import codecs
import csv
import pandas as pd
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import nltk.corpus
from nltk.text import Text
from sklearn.utils import shuffle
from nltk.tokenize import TweetTokenizer
from sklearn.metrics import accuracy_score
from nltk.stem.isri import ISRIStemmer
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import utils
import csv
from tqdm import tqdm
import multiprocessing
import nltk
# from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
from sklearn import preprocessing
from nltk.stem.isri import ISRIStemmer
tknzr = TweetTokenizer()


In [ ]:
import csv
from sklearn import svm
from sklearn.svm import SVC
import pandas as pd
import itertools
import seaborn as sn
import matplotlib.pyplot as plt
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import nltk.corpus
from nltk.text import Text
from sklearn.utils import shuffle
from nltk.tokenize import TweetTokenizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from nltk.stem.isri import ISRIStemmer

In [ ]:
# remove harakat
def remove_harakat(in_data):
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ّ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    out_data = re.sub(noise, '', in_data)
    return out_data


# substitute some characters with others
def substitute_letters(in_data):
    out_data = re.sub("[إأٱآا]", "ا", in_data)
    out_data = re.sub("ى", "ي", out_data)
    out_data = re.sub("ؤ", "ء", out_data)
    out_data = re.sub("ئ", "ء", out_data)
    out_data = re.sub("ة", "ه", out_data)
    out_data = re.sub('URL','يورل', out_data) # this will substitute URL with arabic equivilant
    return(out_data)

# remove everything but characters, spaces and hashtags
def remove_symbols(in_data):
    pattern=r'\\.|[^\w\s]'
    out_data= re.sub(pattern,' ', in_data)
    out_data=re.sub(r'\d',' ',out_data)
    return out_data

# remove multiple spaces
def remove_spaces(in_data):
    pattern=r'\n|\r|\t|(\s+)'
    out_data= re.sub(pattern,' ', in_data)
    return out_data

# remove common stopwords
def remove_stopwords(in_data):
    in_data = tknzr.tokenize(in_data)
    out_data=[w for w in in_data if not w in stop_words ]
    result = " "
    result = result.join(out_data)
    return result

# remove duplicate consecutive characters
# don't allow more than 2 repetiation for one character at a time
def remove_duplicate_characters(in_data):
    return re.compile(r"(.)\1{2,}", re.IGNORECASE).sub("\\1", in_data)

# stemming arabic text using ISRIStemmer
def light_stem1(text):
  words = text.split()
  result = list()
  stemmer = ISRIStemmer()
  for word in words:
    word= stemmer.stem(word)
    result.append(word)

  return ' '.join(result)

def light_stem2(text):
  words = text.split()
  result = list()
  stemmer = ISRIStemmer()
  for word in words:
    word = stemmer.norm(word, num=1)      # remove diacritics which representing Arabic short vowels
    if not word in stemmer.stop_words:    # exclude stop words from being processed
      word = stemmer.pre32(word)        # remove length three and length two prefixes in this order
      word = stemmer.suf32(word)        # remove length three and length two suffixes in this order
      word = stemmer.waw(word)          # remove connective ‘و’ if it precedes a word beginning with ‘و’
      word = stemmer.norm(word, num=2)  # normalize initial hamza to bare alif
    result.append(word)
  return ' '.join(result)

def light_stem3(text):
  ArListem = ArabicLightStemmer()
  words = text.split()
  result = list()
  for word in words:
    word= ArListem.light_stem(word)
    result.append(word)

  return ' '.join(result)


# remove english characters
def remove_english_characters(in_data):
    pattern=r'[A-Za-z]'
    out_data= re.sub(pattern,'', in_data)
    return out_data

## Misc things
import codecs
pattern=r'\r\n|\n'
stop_words_file=codecs.open('/content/drive/My Drive/OffensEval 2020/Source Code/Arabic- StopWords.txt','r','utf-8')
stop_words=[]
for line in stop_words_file:
    new_line=re.sub(pattern,'',line)
    stop_words.append(new_line)
stop_words_file.close()

In [ ]:
# normalize text by using a set of functions
def normalize_text(in_data, with_stemming=False):
        clean_data=remove_harakat(in_data)# remove harakat
        clean_data=remove_symbols(clean_data)# we didn't remove # because we're going to make use of them
        clean_data=substitute_letters(clean_data)# substitute some letters with others
        clean_data= remove_english_characters(clean_data) # remove english characters

        clean_data=remove_stopwords(clean_data)# remove stop words from our text before removing harakat
        if with_stemming==True:
            clean_data=light_stem2(clean_data)
        clean_data=remove_spaces(clean_data) # remove spaces between words
        clean_data=remove_duplicate_characters(clean_data) # remove duplicate characters
        return clean_data

In [ ]:
import re
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
# get  X_train and Y _train and X_test and Y_test
'''

'''

labels = []
def read1(file):
    X = []
    Y = []
    with open(file ,encoding='utf-8') as fp:
        line = fp.readline()
        while line:
            if line == "nan" or line == "\n":
                continue
            else :
                if "NOT_HS" in line:
                    # Label 1
                    line = normalize_text(line)
                    X.append(line)
                    Y.append(0)
                elif "HS" in line:
                    line = normalize_text(line)
                    X.append(line)
                    Y.append(1)
            line = fp.readline()
    fp.close()
    return X,Y



In [ ]:

x_train , y_train = read1("/content/drive/My Drive/OffensEval 2020/Data/offensive_eval_subtask_B/training set.txt" )
print(len(x_train))
x_val  , y_val = read1("/content/drive/My Drive/OffensEval 2020/Data/offensive_eval_subtask_B/Development set.txt")
print(len(x_val))

In [ ]:
import imblearn
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
# down_sampling
smote = SMOTE(ratio='minority')
x_train, y_train = smt.fit_sample(x_train, y_train)



In [ ]:
from imblearn.under_sampling import ClusterCentroids
print
cc = ClusterCentroids(ratio={0: 10})
x_train, y_train = cc.fit_sample(x_train, y_train)


In [ ]:
# under_sampling and over_sampling
import imblearn
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
smt = SMOTETomek(ratio='auto')
x_train, y_train = smt.fit_sample(x_train, y_train)


In [ ]:
count= 0
for i in range(len(y_train)):
  if y_train[i] == 0:
    count+=1
print(count)
print(len(x_train))
print(len(y_train))
print(x_train[1])

In [ ]:
X_train1, y_train1, pos_num1, neg_num1 = read_data('/content/drive/My Drive/OffensEval 2020/Data/offensive_eval_subtask_A/taskA_cleaned_train_file1.txt')
print(len(y_train1), pos_num1, neg_num1, pos_num1/ len(y_train1))
x_val, y_val, pos_num_val, neg_num_val =read_data('/content/drive/My Drive/OffensEval 2020/Data/offensive_eval_subtask_A/taskA_cleaned_val_file1.txt')
print(len(y_val), pos_num_val, neg_num_val, pos_num_val/len(y_val))
X_train2, y_train2, pos_num2, neg_num2 = read_data('/content/drive/My Drive/OffensEval 2020/Data/offensive_eval_subtask_A/taskA_cleaned_train_youtube_file1.txt')
print(len(y_train2), pos_num2, neg_num2, pos_num2/len(y_train2))

In [ ]:
print(len(X_train2))
def balance(x_data, y_data, num_neg_removed):
  count=0
  new_x_data=[]
  new_y_data=[]
  for x,y in zip(x_data, y_data):
    if count>num_neg_removed:
      new_x_data.append(x)
      new_y_data.append(y)
    else:
      if y==1:
        new_x_data.append(x)
        new_y_data.append(y)
      else: count+=1

  return new_x_data, new_y_data

X_train2, y_train2=balance(X_train2, y_train2, 6000)
print(len(X_train2))

In [ ]:
import random
x_train = X_train2 + X_train1
y_train = y_train2 + y_train1

combined = list(zip(x_train, y_train))
random.shuffle(combined)
x_train, y_train= zip(*combined)

x_train = X_train
print(len(x_train))
print(y_train)
print(len(x_val))
y_train = list(y_train)
print(y_train)



In [ ]:
y_train = list(y_train)
print(y_train)


In [ ]:
print(y_val)